# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846864831980                   -0.70    4.8
  2   -7.852320476563       -2.26       -1.53    1.0
  3   -7.852646211684       -3.49       -2.52    3.2
  4   -7.852646677188       -6.33       -3.37    2.2
  5   -7.852646686310       -8.04       -4.65    1.5
  6   -7.852646686728       -9.38       -5.61    3.0
  7   -7.852646686730      -11.79       -6.33    1.5
  8   -7.852646686730      -13.26       -7.46    2.0
  9   -7.852646686730      -14.45       -8.13    2.2
 10   -7.852646686730   +    -Inf       -9.01    2.2
 11   -7.852646686730   +  -14.75       -9.74    2.2
 12   -7.852646686730      -14.75      -10.87    2.5
 13   -7.852646686730   +  -14.75      -11.61    3.0
 14   -7.852646686730   +    -Inf      -12.61    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846903922259                   -0.70           4.8
  2   -7.852526362818       -2.25       -1.64   0.80    2.0
  3   -7.852636124149       -3.96       -2.73   0.80    1.0
  4   -7.852646501450       -4.98       -3.24   0.80    2.0
  5   -7.852646673454       -6.76       -4.08   0.80    1.0
  6   -7.852646686374       -7.89       -4.80   0.80    1.8
  7   -7.852646686725       -9.45       -5.53   0.80    1.8
  8   -7.852646686729      -11.34       -6.89   0.80    1.5
  9   -7.852646686730      -12.33       -7.02   0.80    3.0
 10   -7.852646686730   +  -15.05       -7.81   0.80    1.0
 11   -7.852646686730   +    -Inf       -8.97   0.80    1.2
 12   -7.852646686730      -15.05       -9.30   0.80    2.5
 13   -7.852646686730   +    -Inf       -9.70   0.80    1.0
 14   -7.852646686730   +    -Inf      -10.77   0.80    1.2
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.395722e+01     3.357735e+00
 * time: 0.38953709602355957
     1     1.224852e+00     1.712618e+00
 * time: 0.5931251049041748
     2    -1.580323e+00     1.925210e+00
 * time: 0.6158449649810791
     3    -3.638169e+00     1.741070e+00
 * time: 0.6482610702514648
     4    -4.767823e+00     1.633621e+00
 * time: 0.6804769039154053
     5    -6.550105e+00     1.101609e+00
 * time: 0.7129080295562744
     6    -7.262372e+00     6.798549e-01
 * time: 0.7456309795379639
     7    -7.558813e+00     4.725440e-01
 * time: 0.7683501243591309
     8    -7.671351e+00     1.509780e-01
 * time: 0.7910919189453125
     9    -7.763530e+00     2.172366e-01
 * time: 0.8139231204986572
    10    -7.809558e+00     6.580967e-02
 * time: 0.8366289138793945
    11    -7.828410e+00     6.342906e-02
 * time: 0.8593940734863281
    12    -7.840512e+00     5.914150e-02
 * time: 0.8822081089019775
    13    -7.845868e+00     5.154915e-02
 * time: 0.904932975

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846690036403                   -0.70    4.8
  2   -7.852292936525       -2.25       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686709                   -2.54
  2   -7.852646686730      -10.69       -5.94
  3   -7.852646686730      -14.35      -12.67


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.0436706273505455e-13
|ρ_newton - ρ_scfv| = 9.003532055925759e-13
|ρ_newton - ρ_dm|   = 8.757682192825914e-10
